In [ ]:
import os
os.chdir("..")

In [ ]:
import numpy as np
from operator import attrgetter, add
from functools import reduce
from tqdm import tqdm

from engine.tsp import TspWrapper

np.random.seed(2137)

In [ ]:
tsp_problem = TspWrapper.from_atsp_full_matrix("data/ftv33.atsp")

In [ ]:
allowed_maximum_groups = 40
perturbation_rate = 0.7
local_leader_limit = 5
global_leader_limit = 20
spider_monkeys_count = 500
global_leader_count = 0

In [ ]:
vertices = tsp_problem.get_vertices()

In [ ]:
spider_monkes = [
    np.random.permutation(vertices) for _ in range(spider_monkeys_count)
]

In [ ]:
class MonkeGroup:

    def __init__(self, monkes, leader):
        self.monkes = monkes
        self.leader = leader
        self.counter = 0

    def __len__(self):
        return len(self.monkes)

In [ ]:
costs = list(map(tsp_problem.calculate_cost, spider_monkes))
global_leader = spider_monkes[np.argmin(costs)]
groups = [MonkeGroup(spider_monkes, global_leader)]

In [ ]:
def permutation_difference(
        perm_1: np.ndarray, perm_2: np.ndarray
    ) -> callable:
        arg_1 = np.argsort(perm_1)
        return arg_1[perm_2]

def decompose_permutation_to_swap_sequence(permutation):
    swaps = []
    permutation = permutation.tolist()
    for i in range(len(permutation)):
        idx = permutation.index(i)
        swaps.append((i, idx))
        permutation[i], permutation[idx] = i, permutation[i]
    return np.array(swaps[-1::-1])

def swap_sequence_to_permutation(ss):
    result = np.arange(np.max(ss) + 1)
    for pos1, pos2 in ss:
        result[pos1], result[pos2] = result[pos2], result[pos1]
    return result

def merge_swap_sequences(ss1, ss2):
    return ss1.tolist() + ss2.tolist()

def apply_swap_sequence(perm, ss):
    perm = perm.copy()
    for so in ss:
        perm[so[1]], perm[so[0]] = perm[so[0]], perm[so[1]]
    return perm

In [ ]:
def choose_in_order(l):
    if len(l) == 0:
        return np.array([])
    length =  np.random.choice(np.arange(1, len(l) + 1))
    idx = np.unique(np.random.choice(np.arange(len(l)), size=length))
    idx = np.sort(idx)
    return l[idx]

choose_in_order(np.array([1, 2, 3, 4, 5, 6, 7, 9]))

In [ ]:
def update_group_1(group: MonkeGroup):
    for i in range(len(group)):
        if np.random.random() >= perturbation_rate:
            monke = group.monkes[i]
            random_monke = group.monkes[np.random.choice(np.arange(len(group)))]

            ss1 = decompose_permutation_to_swap_sequence(permutation_difference(monke, group.leader))
            ss1 = choose_in_order(ss1)
            ss2 = decompose_permutation_to_swap_sequence(permutation_difference(monke, random_monke))
            ss2 = choose_in_order(ss2)
            merged_ss = merge_swap_sequences(ss1, ss2)
            new_monke = apply_swap_sequence(monke, merged_ss)
            if tsp_problem.calculate_cost(new_monke) < tsp_problem.calculate_cost(monke):
                group.monkes[i] = new_monke

In [ ]:
def update_group_2(group: MonkeGroup, global_leader):
    for i in range(len(group)):
        monke = group.monkes[i]
        prob = .9*tsp_problem.calculate_cost(global_leader)/tsp_problem.calculate_cost(monke) + .1
        if np.random.random() <= prob:
            random_monke = group.monkes[np.random.choice(np.arange(len(group)))]
            ss1 = decompose_permutation_to_swap_sequence(permutation_difference(monke, global_leader))
            ss1 = choose_in_order(ss1)
            ss2 = decompose_permutation_to_swap_sequence(permutation_difference(monke, random_monke))
            ss2 = choose_in_order(ss2)
            merged_ss = merge_swap_sequences(ss1, ss2)
            new_monke = apply_swap_sequence(monke, merged_ss)
            if tsp_problem.calculate_cost(new_monke) < tsp_problem.calculate_cost(monke):
                group.monkes[i] = new_monke

In [ ]:
def resolve_leaders(groups):
    global global_leader_count
    global global_leader

    # Resolve local
    for group in groups:
        monkes_cost = list(map(tsp_problem.calculate_cost, group.monkes))
        leader_cost = tsp_problem.calculate_cost(group.leader)
        min_cost_monke_idx = np.argmin(monkes_cost)
        if monkes_cost[min_cost_monke_idx] < leader_cost:
            group.leader = group.monkes[min_cost_monke_idx]
            group.counter = 0
        else:
            group.counter += 1

    # Resolve global
    local_leaders = list(map(attrgetter("leader"), groups))
    leaders_costs = list(map(tsp_problem.calculate_cost, local_leaders))
    min_cost_leader_idx = np.argmin(leaders_costs)
    if leaders_costs[min_cost_leader_idx] < tsp_problem.calculate_cost(global_leader):
        global_leader = groups[min_cost_leader_idx].leader
        global_leader_count = 0
    else:
        global_leader_count += 1 

In [ ]:
def decision_phase_1(groups):
    global local_leader_limit
    
    for group in groups:
        if group.counter > local_leader_limit:
            group.counter = 0
            if np.random.uniform() < perturbation_rate:
                for i in range(len(group)):
                    monke = group.monkes[i]
                    ss1 = decompose_permutation_to_swap_sequence(permutation_difference(monke, global_leader))
                    ss1 = choose_in_order(ss1)
                    ss2 = decompose_permutation_to_swap_sequence(permutation_difference(monke, group.leader))
                    ss2 = choose_in_order(ss2)
                    new_monke = apply_swap_sequence(monke, ss1)
                    new_monke = apply_swap_sequence(new_monke, ss2)
            else:
                group.monkes = [
                    np.random.permutation(vertices) for _ in range(spider_monkeys_count)
                ]
            best_monke_in_group_idx = np.argmin(list(map(tsp_problem.calculate_cost, group.monkes)))
            group.leader = group.monkes[best_monke_in_group_idx]

In [ ]:
def decision_phase_2(groups):
    global global_leader
    global global_leader_count

    if global_leader_count > global_leader_limit:
        global_leader_count = 0
        if len(groups) < allowed_maximum_groups:
            group = groups.pop(0)
            group_new_1 = group.monkes[0:len(group)//2]
            group_new_2 = group.monkes[len(group)//2 + 1:-1]
            group_new_1 = MonkeGroup(group_new_1, group_new_1[np.argmin(list(map(tsp_problem.calculate_cost, group_new_1)))])
            group_new_2 = MonkeGroup(group_new_2, group_new_2[np.argmin(list(map(tsp_problem.calculate_cost, group_new_2)))])
            groups.append(group_new_1)
            groups.append(group_new_2)
        else:
            consolidated_group_monkes = reduce(add, map(attrgetter("monkes"), groups))
            leader = global_leader
            groups = [MonkeGroup(consolidated_group_monkes, leader)]
    return groups

In [ ]:
# nazwa: znany najlepszy - otrzymany
# br17: 39 - 36
# ft53: 6905 - 8050 (15 min)
# rbg443: 2720 - 4749 (13 min)
# kro124: 36230 - 5940 (11 min)
# ftv33: 1286 - 1560 (7 min)

In [ ]:
best_solution = global_leader
iteration_counter = 0
iter = tqdm(range(1000))
for i in iter:
    # Algorithm 2.1
    for group in groups:
        update_group_1(group)
        update_group_2(group, global_leader)
    
    # Algorithm 2.2
    resolve_leaders(groups)

    # Algorithm 2.3
    decision_phase_1(groups)
    groups = decision_phase_2(groups)

    best_solution = best_solution if tsp_problem.calculate_cost(best_solution) < tsp_problem.calculate_cost(global_leader) else global_leader
    iter.set_description(f"{tsp_problem.calculate_cost(best_solution)}")

    iteration_counter += 1
    